# Notebook for EDV-ESV data annotaiton generation

## Volume calculation

In [ ]:
import pandas as pd
import numpy as np

# Load your file
file_path = 'your/path/to/VolumeTracings.csv'  # Replace with your file path
data = pd.read_csv(file_path)


In [3]:

# Function to calculate the Euclidean distance between two points
def euclidean_distance(x1, y1, x2, y2):
    return np.sqrt((x2 - x1) ** 2 + (y2 - y1) ** 2)

In [4]:
def volumn_calculation(data):
    volumns = []
    for (file_name, frame), group in data.groupby(['FileName', 'Frame']):

        # Calculate average height between slices
        avg_h = euclidean_distance(group.iloc[0].X1, group.iloc[0].Y1, group.iloc[0].X2, group.iloc[0].Y2)/(len(group)-1)
        volumn = 0
        for index, row in group.iloc[1:].iterrows():

            # Calculate radius and area
            r = euclidean_distance(row.X1, row.Y1, row.X2, row.Y2)/2
            s = np.pi*r*r
            volumn += s*avg_h
        volumns.append(volumn)
    return volumns

In [7]:
result = volumn_calculation(data)


In [8]:
unique_combination = data[['FileName', 'Frame']].drop_duplicates().reset_index(drop=True)
unique_combination['Volumn'] = pd.DataFrame(result)

In [11]:
unique_combination.to_csv('calculated_volumns.csv', index=False)

## EDV-ESV identification

In [12]:
#load annotation file
annotation_file_path = 'calculated_volumns.csv'
annotation = pd.read_csv(annotation_file_path)

In [13]:
# Label EDV and ESV based on maximum and minimum volume per FileName
annotation['Label'] = annotation.groupby("FileName")["Volumn"].transform(
    lambda x: ["EDV" if v == x.max() else "ESV" for v in x]
)

In [15]:
annotation.to_csv('labeled_volumns.csv', index=False)